## Preprocess data

In [1]:
import cv2
import numpy as np
import pandas as pd

## 將蒐集好的數字圖片資料庫傳化成深度學習使用的訓練資料,需要:
#### 1.一般圖片有RGB,3個CHANNEL,用於字體辨識的前提下,字的形體才是主要因素而不是顏色,因此必須將原本有三通道的圖片轉為單通道的灰階圖
#### 2.將圖片轉成長度為"長乘以寬"的1D陣列,或以實例舉例: 一張 28X28的圖片,轉成1D陣列後的形體會是有784個數值的陣列
#### 3.將每個數字圖片對應的label轉成ONE_HOT_ENCODING形式

In [2]:
#取得所有圖片資料,並且由圖片編號排序
import os
def ls(root):
    ior_folder = root
    files = os.listdir(ior_folder)
    files = sorted(files, key = lambda x:int(x[3:].split('.')[0]), reverse=False)
    return files

## DEMO 圖片轉換過程
以下過程是將一28X28圖片轉為1D陣列的驗證

In [3]:
image = cv2.imread('ior/ior0.jpg', 0) #讀取圖片為灰階圖
print('Before convertion: Row-{}, Column-{}'.format(image.shape[0], image.shape[1])) #顯示原本陣列形狀,原始為(28, 28)
image = image.reshape(1, 28**2) #轉為1D陣列
print('Before convertion: Row-{}, Column-{}'.format(image.shape[0], image.shape[1])) #顯示轉換後陣列形狀,為(1, 784)

Before convertion: Row-28, Column-28
Before convertion: Row-1, Column-784


# Dataset class

In [24]:
import cv2
import numpy as np
import pandas as pd

class digitsDataset:
    res = 28
    root = 'ior/'
    training = 0.9
    def __init(self):
        pass
    
    def ls(self):
        files = os.listdir(self.root)
        files = sorted(files, key = lambda x:int(x[3:].split('.')[0]), reverse=False)
        return files

    #讀取所有圖片檔案,轉為1D陣列
    def load_image(self):
        files = self.ls()
        X = []

        for index, obj in enumerate(files):
            image = cv2.imread((self.root + obj), 0)
            flatten = image.reshape(self.res**2,)
            X.append(flatten)
        return np.array(X)

    #當學習數值為離散型或為分類任務時,使用one hot encoding
    def one_hot(self, columns, value):
        output = np.zeros((1, columns), dtype=np.int8)
        output[0][value] = 1
        return output

    #每張圖片所對應之label,轉為one hot型態
    def load_labels(self):
        file = 'labels.csv'
        values = pd.read_csv(file).values
        rows = len(values)
        columns = np.max(values) + 1
        output = np.zeros((rows,columns))
        for index, number in enumerate(values):
            one_hot_encoding = self.one_hot(columns, number)
            output[index] = one_hot_encoding
        return output

    def LOAD_DATA(self, one_hot = True):
        if one_hot == True:
            X = self.load_image()
            y = self.load_labels()
            
            index = int((len(X) + len(y))/2*self.training)
            
            train_x = X[0:index,:]
            train_y = y[0:index,:]
            test_x = X[index:,:]
            test_y = y[index:,:]
            
            return (train_x, train_y, test_x, test_y)

In [25]:
train_x, train_y, test_x, test_y = digitsDataset().LOAD_DATA()

In [26]:
train_y.dtype

dtype('float64')

In [27]:
from tflearn.datasets.mnist import load_data

mnist = load_data()


Succesfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting mnist/train-images-idx3-ubyte.gz
Succesfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting mnist/train-labels-idx1-ubyte.gz
Succesfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist/t10k-images-idx3-ubyte.gz
Succesfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [11]:
train_x = train_x.astype(np.float32)
train_y = train_y.astype(np.float32)


x = tf.placeholder(tf.int8, name="x", shape=[None, 784])

W = tf.Variable(tf.random_uniform([784, 10], -1, 1), name="W")

b = tf.Variable(tf.zeros([10]), name="biases")

output = tf.matmul(x, W) + b

loss = tf.reduce_mean(train_y - output)

init_op = tf.initialize_all_variables()



sess = tf.Session()

sess.run(init_op)

feed_dict = {"x" : train_x}

value = sess.run(output, feed_dict= feed_dict)
print(value)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


TypeError: Cannot interpret feed_dict key as Tensor: The name 'x' refers to an Operation, not a Tensor. Tensor names must be of the form "<op_name>:<output_index>".

In [14]:
train_x.dtype

dtype('float32')

In [13]:
train_y.shape

(110, 10)

In [17]:
cv2.imshow('', train_x[0].astype(np.int8).reshape(28, 28))
cv2.waitKey()

-1

In [10]:
import tensorflow as tf

train_x = train_x.astype(np.float32)
train_y = train_y.astype(np.float32)


x = tf.placeholder(tf.float32, name="x", shape=[None, 784])
W = tf.Variable(tf.random_uniform([784,10], -1, 1), name="W")
multiply = tf.matmul(x, W)

y = tf.nn.softmax(multiply)
loss = tf.reduce_mean(tf.square(train_y - y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(50):
        sess.run(train_step, feed_dict={x:train_x,y:train_y})
        print(sess.run(loss))


InternalError: Blas SGEMM launch failed : a.shape=(110, 10), b.shape=(784, 10), m=110, n=784, k=10
	 [[Node: gradients_1/MatMul_1_grad/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=true, _device="/job:localhost/replica:0/task:0/gpu:0"](gradients_1/Softmax_1_grad/mul_1, W/read)]]

Caused by op 'gradients_1/MatMul_1_grad/MatMul', defined at:
  File "C:\Users\w1243\AppData\Local\conda\conda\envs\tensorflow_gpu_env\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\w1243\AppData\Local\conda\conda\envs\tensorflow_gpu_env\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\w1243\AppData\Local\conda\conda\envs\tensorflow_gpu_env\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\w1243\AppData\Local\conda\conda\envs\tensorflow_gpu_env\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\w1243\Ap

In [14]:
import tensorflow as tf

In [23]:
def add_layer(input, in_size, out_size, activation_function=None):
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_biases = tf.matmul(input, Weights) + biases
    
    if activation_function is None:
        output = Wx_plus_biases
    else:
        output = activation_function(output)
    return output

In [28]:
input_to_hidden = add_layer(train1, 110, 200, activation_function=tf.nn.relu)
hidden_to_hidden2 = add_layer(input_to_hidden, 200, 30, activation_function=tf.nn.relu)
y = add_layer(hidden_to_hidden2, 30, 10, activation_function=tf.nn.softmax)
loss = tf.reduce_mean(tf.square(train_y - y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    tf.run(init)
    for i in range(50):
        tf.run(train_step)
        print(loss)

ValueError: Dimensions must be equal, but are 784 and 110 for 'MatMul_6' (op: 'MatMul') with input shapes: [110,784], [110,200].

In [6]:
train_x.shape

(110, 784)

In [7]:
train_y.shape

(110, 10)

In [8]:
# Import Numpy, TensorFlow, TFLearn, and MNIST data
import tensorflow as tf
import tflearn

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [9]:
# Define the neural network
def build_model():
    # This resets all parameters and variables, leave this here
    tf.reset_default_graph()
    
    #### Your code ####
    # Include the input layer, hidden layer(s), and set how you want to train the model
    net = tflearn.input_data([None, 784])
    net = tflearn.fully_connected(net, 200, activation='ReLU')
    net = tflearn.fully_connected(net, 10)
    net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss='categorical_crossentropy')
    # This model assumes that your network is named "net"    
    model = tflearn.DNN(net)
    return model

In [10]:
model = build_model()

In [11]:
model.fit(train_x, train_y, validation_set=0.1)

---------------------------------
Run id: 1SN7JG
Log directory: /tmp/tflearn_logs/
---------------------------------
Training samples: 99
Validation samples: 11
--


InternalError: Blas SGEMM launch failed : a.shape=(64, 784), b.shape=(784, 200), m=64, n=200, k=784
	 [[Node: FullyConnected/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](_recv_InputData/X_0/_13, FullyConnected/W/read)]]

Caused by op 'FullyConnected/MatMul', defined at:
  File "C:\Users\w1243\AppData\Local\conda\conda\envs\tensorflow_gpu_env\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\w1243\AppData\Local\conda\conda\envs\tensorflow_gpu_env\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\w1243\AppData\Local\conda\conda\envs\tensorflow_gpu_env\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\w1243\AppData\Local\conda\conda\envs\tensorflow_gpu_env\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\w1243\AppData\Local\co

In [ ]:
model.fit(train_x, train_y, validation_set=0.1,  batch_size=5, n_epoch=20)

In [ ]:
train_x.shape

In [85]:
train_y.shape

(110, 10)